In [2]:
import pandas as pd
import glob
import os

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
#from google.colab import drive

from random import sample 

In [ ]:
#drive.mount('/content/drive/', True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [4]:
input_df = pd.read_csv("order_brush_order.csv")

In [5]:
input_df.shape

(222750, 4)

In [6]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222750 entries, 0 to 222749
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   orderid     222750 non-null  int64 
 1   shopid      222750 non-null  int64 
 2   userid      222750 non-null  int64 
 3   event_time  222750 non-null  object
dtypes: int64(3), object(1)
memory usage: 6.8+ MB


In [7]:
input_df.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [8]:
input_df = input_df.assign(event_time=pd.to_datetime(input_df['event_time']))
input_df.dtypes

orderid                int64
shopid                 int64
userid                 int64
event_time    datetime64[ns]
dtype: object

In [9]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222750 entries, 0 to 222749
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   orderid     222750 non-null  int64         
 1   shopid      222750 non-null  int64         
 2   userid      222750 non-null  int64         
 3   event_time  222750 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 6.8 MB


In [10]:
def prepare_output(df):
  df = df \
    .shopid \
    .unique()
  
  df = pd.DataFrame(df)
  df = df.assign(result=np.nan)
  df = df.rename(columns={0:'shopid'})
  df = df.set_index('shopid')
  return df

base_output_df = input_df.pipe(prepare_output)
base_output_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18770 entries, 93950878 to 147118226
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   result  0 non-null      float64
dtypes: float64(1)
memory usage: 293.3 KB


In [11]:
def get_less_than_threshold_user(df, threshold=1):
  df = df \
    .groupby('shopid') \
    .count() \
    .userid \
    .to_frame()

  df = df[df.userid <= threshold]
  
  df = df.assign(result=0)
  df = df [['result']]
  return df

small_userid_df = input_df.pipe(get_less_than_threshold_user)
small_userid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7085 entries, 10009 to 215435223
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   result  7085 non-null   int64
dtypes: int64(1)
memory usage: 110.7 KB


In [12]:
def get_less_than_threshold_hour(df, threshold=pd.Timedelta(3, unit='h')):
  df = df \
    .groupby('shopid') \
    .agg({'event_time':[np.max, np.min]})

  df = df.assign(time_diff=df.event_time.amax - df.event_time.amin)
  df = df[df.time_diff <= threshold]

  df = df.assign(result=0)
  df = df[['result']]
  df.columns = df.columns.get_level_values(0)
  return df

small_hour_df = input_df.pipe(get_less_than_threshold_hour)
small_hour_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7445 entries, 10009 to 215435223
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   result  7445 non-null   int64
dtypes: int64(1)
memory usage: 116.3 KB


In [16]:
def get_highest_user(df):
  df = df.sort_values(by='event_time')
  df = df \
    .groupby(['shopid', 'userid']) \
    .rolling('1D', on='event_time') \
    .orderid \
    .count() \
    .to_frame()
  
  df = df \
    .assign(rnk=df.groupby(['shopid']).orderid.rank(method='dense', ascending=False))
  
  df = df[df.rnk == 1]
  return df

highest_user_df = input_df.pipe(get_highest_user)
highest_user_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 73712 entries, (10009, 196962305, Timestamp('2019-12-27 03:06:50')) to (215435223, 79515106, Timestamp('2019-12-31 18:34:56'))
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   orderid  73712 non-null  float64
 1   rnk      73712 non-null  float64
dtypes: float64(2)
memory usage: 3.6 MB


In [ ]:
def get_brusher(df):
  df = df.reset_index(level = ['shopid', 'userid', 'event_time'])
  df = df.drop('event_time', axis = 1)
  df.loc[df.orderid <= 3, 'userid'] = 0
  df = df.drop_duplicates()

  df = df \
    .groupby('shopid') \
    .userid \
    .apply(list) \
    .to_frame()

  df = df \
    .assign(result=df.userid.apply(lambda x: ' & '.join(map(str, x)))) \
    .assign(user_count=df.userid.apply(lambda x: len(x)))

  df = df[['result']]
  
  return df

brusher_df = highest_user_df.pipe(get_brusher)
brusher_df.info()

In [ ]:
def join_output(df, temp_df):
  df = df.combine_first(temp_df)
  return df

def normalize_float(df):
  df = df \
    .assign(is_float=output_df.result.apply(lambda x: isinstance(x, float)))

  df.loc[x_df.is_float, 'result'] = df.loc[x_df.is_float, 'result'].astype(int)
  df = df[['result']]
  return df

output_df = base_output_df \
  .pipe(join_output, small_hour_df) \
  .pipe(join_output, small_userid_df) \
  .pipe(join_output, brusher_df) \
  .pipe(normalize_float)

output_df.info()

In [ ]:
output_df

In [ ]:
output_df.to_csv('output.csv')

In [ ]:
!tail output.csv

In [ ]:
input_df